In [3]:
import itertools
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

In [11]:
df = pd.read_csv("https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv")
df.dtypes

SEVERITYCODE        int64
X                 float64
Y                 float64
OBJECTID            int64
INCKEY              int64
COLDETKEY           int64
REPORTNO           object
STATUS             object
ADDRTYPE           object
INTKEY            float64
LOCATION           object
EXCEPTRSNCODE      object
EXCEPTRSNDESC      object
SEVERITYCODE.1      int64
SEVERITYDESC       object
COLLISIONTYPE      object
PERSONCOUNT         int64
PEDCOUNT            int64
PEDCYLCOUNT         int64
VEHCOUNT            int64
INCDATE            object
INCDTTM            object
JUNCTIONTYPE       object
SDOT_COLCODE        int64
SDOT_COLDESC       object
INATTENTIONIND     object
UNDERINFL          object
WEATHER            object
ROADCOND           object
LIGHTCOND          object
PEDROWNOTGRNT      object
SDOTCOLNUM        float64
SPEEDING           object
ST_COLCODE         object
ST_COLDESC         object
SEGLANEKEY          int64
CROSSWALKKEY        int64
HITPARKEDCAR       object
dtype: objec

In [13]:
df['SEVERITYDESC'].value_counts().to_frame() # choosing columns to use from the dataframe

,SEVERITYDESC
Property Damage Only Collision,136485
Injury Collision,58188


In [14]:
ax = df_Severity.plot.bar(y = 'SEVERITYCODE', rot = 0)

NameError: name 'df_Severity' is not defined

In [15]:
df_Conditions.balanced['COLLISIONTYPE'].value_counts().plot(kind='bar')

NameError: name 'df_Conditions' is not defined

In [16]:
df_Conditions.balanced['WEATHER'].value_counts().plot(kind='bar')

NameError: name 'df_Conditions' is not defined

In [17]:
df_Conditions.balanced['INATTENTIONIND'].value_counts().plot(kind='bar')

NameError: name 'df_Conditions' is not defined

In [18]:
df_Conditions.balanced['UNDERINFL'].value_counts().plot(kind='bar')

NameError: name 'df_Conditions' is not defined